
References:  
1. [Time series classification baseline with CNN](https://github.com/cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline/blob/master/FCN.py)
2. GAN implementation examples:  
    1. [Image Captioning with Attention](https://github.com/tensorflow/tensorflow/blob/r1.13/tensorflow/contrib/eager/python/examples/generative_examples/image_captioning_with_attention.ipynb)
    2. [Convolutional VAE](https://github.com/tensorflow/tensorflow/blob/r1.13/tensorflow/contrib/eager/python/examples/generative_examples/cvae.ipynb)


# Mount Google Drive as remote drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/TSI/Master\'s thesis/space-type-recognition')


In [ ]:
if 'COLAB_TPU_ADDR' in os.environ:
  TPU_ENABLED = True
  TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print('TPU address is', TPU_WORKER)

  with tf.Session(TPU_WORKER) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  display(devices)
else:
  TPU_ENABLED = False
  
nb_epochs = 2000

%connect_info


# Install required libraries

In [ ]:
import sys
!{sys.executable} -m pip install tensorflow-gpu==1.13.1 tensorboard==1.13.1

# Import base libraries


In [1]:
import tensorflow as tf
import tensorflow.keras.utils as np_utils
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard
from tensorflow.keras import layers, regularizers, optimizers
from tensorflow.contrib.gan.python.losses.python.losses_impl import wasserstein_discriminator_loss, wasserstein_generator_loss
from timeseries.helper import plot_model_recursive
from timeseries.data import load_ucr_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from timeseries.data import Dataset
from timeseries.transformers import AaeClustering, AdversarialAutoencoder

import numpy as np
import os
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from functools import reduce

from sklearn.preprocessing import minmax_scale, scale

import sys

display(sys.version)
display(tf.VERSION)


# Define Adversarial Autoencoder (AAE) Model and Blocks

Originally, AAE is described by [https://arxiv.org/abs/1511.05644](https://arxiv.org/abs/1511.05644).
More detailed description can be found [https://pdfs.semanticscholar.org/7989/533d5169904998271ecfa5377c21404c7348.pdf](https://pdfs.semanticscholar.org/7989/533d5169904998271ecfa5377c21404c7348.pdf)


See Tensorflow documentation to [create custom Models](https://www.tensorflow.org/alpha/guide/keras/custom_layers_and_models#putting_it_all_together_an_end-to-end_example) 

In [2]:
def create_encoder(
        input_shape,
        #classes_num,
        code_len,
        name='encoder',
        reg=regularizers.l1_l2(1e-7, 0),
        units=256,
        renorm=True):
    
    if input_shape[0] > units:
        raise Exception('Encoder input length is %s but expecting less then %s units size. Otherwise autoencoder will not convergence by reconstruction loss' % (input_shape[0], units))
    inputs = layers.Input(input_shape, name=name + "_input")
    conv = inputs
    
    pad_units = units - input_shape[0]
    if pad_units > 0:
        conv = layers.ZeroPadding1D(padding=(0, (pad_units)))(conv)
    
    # conv = layers.BatchNormalization(renorm=renorm)(conv)
        
    conv = layers.Conv1D(int(units / 4), 8, padding='same')(conv)
    conv = layers.BatchNormalization(renorm=renorm)(conv)
    # conv = layers.Activation('relu')(conv)
    conv = layers.LeakyReLU(0.2)(conv)
    
    conv = layers.Conv1D(int(units / 4), 8, padding='same')(conv)
    conv = layers.BatchNormalization(renorm=renorm)(conv)
    # conv = layers.Activation('relu')(conv)
    conv = layers.LeakyReLU(0.2)(conv)
    
    conv = layers.MaxPooling1D()(conv)
    
    conv = layers.Conv1D(int(units / 2), 5, padding='same')(conv)
    conv = layers.BatchNormalization(renorm=renorm)(conv)
    # conv = layers.Activation('relu')(conv)
    conv = layers.LeakyReLU(0.2)(conv)
    
    conv = layers.Conv1D(int(units / 2), 5, padding='same')(conv)
    conv = layers.BatchNormalization(renorm=renorm)(conv)
    # conv = layers.Activation('relu')(conv)
    conv = layers.LeakyReLU(0.2)(conv)
    
    conv = layers.MaxPooling1D()(conv)
    
    conv = layers.Conv1D(units, 3, padding='same')(conv)
    conv = layers.BatchNormalization(renorm=renorm)(conv)
    # conv = layers.Activation('relu')(conv)
    conv = layers.LeakyReLU(0.2)(conv)
    
    conv = layers.MaxPooling1D()(conv)
    
    conv = layers.Conv1D(units, 3, padding='same')(conv)
    conv = layers.BatchNormalization(renorm=renorm)(conv)
    conv = layers.LeakyReLU(0.2)(conv)
    
    if False:
        conv = layers.GlobalAveragePooling2D()(conv)
        
        conv = layers.Flatten()(conv)
        
        conv = layers.Dense(code_len)(conv)
        # conv = layers.Dense(code_len, kernel_regularizer=reg)(conv)
        # TODO: Dropout?
        # TODO: Activation sigmoid?
        conv = layers.Activation('relu')(conv)
        # conv = layers.Activation('sigmoid')(conv)
    
    output = conv

    return tf.keras.Model(inputs, output, name=name)
            
        
        
def create_decoder(
        input_shape,
        output_shape,
        #classes_num,
        code_len,
        name='decoder',
        units=256,
        renorm=True):
    
    inputs = layers.Input(input_shape, name=name + "_input")
    conv = inputs
    if False:
        inputs = layers.Input((code_len, ), name=name + "_input")
        conv = inputs
        
        conv = layers.Dense(units=output_shape[0], activation=tf.nn.relu)(conv)
        conv = layers.Reshape(target_shape=output_shape)(conv)
    
    conv = layers.Conv1D(units, 3, padding='same')(conv)
    conv = layers.BatchNormalization(renorm=renorm)(conv)
    # conv = layers.Activation('relu')(conv)
    conv = layers.LeakyReLU(0.2)(conv)
    conv = layers.UpSampling1D(size=2)(conv)
    
    conv = layers.Conv1D(int(units / 2), 5, padding='same')(conv)
    conv = layers.BatchNormalization(renorm=renorm)(conv)
    # conv = layers.Activation('relu')(conv)
    conv = layers.LeakyReLU(0.2)(conv)
    conv = layers.UpSampling1D(size=2)(conv)
    
    conv = layers.Conv1D(int(units / 4), 8, padding='same')(conv)
    conv = layers.BatchNormalization(renorm=renorm)(conv)
    conv = layers.LeakyReLU(0.2)(conv)
    conv = layers.UpSampling1D(size=2)(conv)
    
    # conv = layers.Conv2DTranspose(1, 251, 1, padding='same')(conv)
    
    conv = layers.Conv1D(output_shape[1], 8, padding='same')(conv)
    conv = layers.BatchNormalization(renorm=renorm)(conv)
    # last layer with linear or sigmoid activation
    conv = layers.Activation('linear')(conv)
    # conv = layers.Activation('sigmoid')(conv)
    
    if conv.shape[1] > output_shape[0]: 
        conv = layers.Cropping1D(cropping=(0, conv.shape[1].value - output_shape[0]))(conv)
    else:
        conv = layers.ZeroPadding1D(padding=(0, output_shape[0] - conv.shape[1].value))(conv)
    
    output = conv
    return tf.keras.Model(inputs, output, name=name)

    
def create_generator_softmax(
        inputs,
        generator,
        classes_num,
        name='generator_softmax',
        trainable=True,
        renorm=True):
    
        generator = layers.Dense(512, trainable=trainable)(generator(inputs))
        generator = layers.BatchNormalization(renorm=renorm)(generator)
        generator = layers.LeakyReLU(0.2)(generator)
        generator = layers.Dense(classes_num, trainable=trainable)(generator)
        generator = layers.BatchNormalization(momentum=0.8, renorm=renorm)(generator)
        output = layers.Softmax(trainable=trainable)(generator)
        
        return tf.keras.Model(inputs, output, name=name)


def create_generator_linear(
        inputs,
        generator,
        code_len,
        name='generator_linear',
        trainable=True,
        renorm=True,
        **kwargs):
    
        generator = layers.Dense(code_len, trainable=trainable)(generator(inputs))
        generator = layers.BatchNormalization(momentum=0.8, renorm=renorm)(generator)

        return tf.keras.Model(inputs, generator, name=name)

def create_generator_linear_softmax(
        inputs,
        generator,
        code_len,
        classes_num,
        dense_size=1024,
        name='generator',
        trainable=True,
        renorm=True,
        reg=regularizers.l1_l2(1e-7, 1e-7),
):
    generator = layers.Flatten()(generator(inputs))
    generator = layers.Dense(dense_size, trainable=trainable, kernel_regularizer=reg)(generator)
    generator = layers.BatchNormalization(renorm=renorm)(generator)
    generator = layers.LeakyReLU(0.2)(generator)
    generator = tf.keras.Model(inputs, generator, name=name)
    
    generator_linear = create_generator_linear(inputs, generator, code_len)
    
    generator_softmax = create_generator_softmax(inputs, generator, classes_num)
    
    return generator_linear, generator_softmax

def create_generator_concatenate(
        inputs,
        generator_softmax,
        generator_linear,
        decoder,
        classes_num,
        code_len,
        name='generator_concatinate'):
    
        autoencoder = layers.Concatenate()([generator_linear(inputs), generator_softmax(inputs)])
        
        autoencoder = layers.Dense(code_len + classes_num)(autoencoder)
        autoencoder = layers.LeakyReLU(0.2)(autoencoder)
        
        autoencoder = layers.Dense(code_len + classes_num)(autoencoder)
        autoencoder = layers.LeakyReLU(0.2)(autoencoder)
        
        autoencoder = layers.Dense(reduce(lambda a, b: a*b, decoder.input_shape[1:]))(autoencoder)
        # TODO: Do we need ReLU here? In original implementation it has linear activation
        # autoencoder = layers.LeakyReLU(0.2)(autoencoder)
        # autoencoder = layers.Dense(code_len)(autoencoder)
        # autoencoder = layers.LeakyReLU(0.2)(autoencoder)
        autoencoder = layers.Reshape(decoder.input_shape[1:])(autoencoder)
        output = decoder(autoencoder)
        
        return tf.keras.Model(inputs=inputs, outputs=output, name=name)
    
    
def create_discriminator(
        latent_dim,
        output_dim=1,
        hidden_dim=256,
        reg=regularizers.l1_l2(1e-7, 1e-7),
        name='discriminator',
        **kwargs):
    
        with K.name_scope('Discriminator'):
            inputs = layers.Input((latent_dim,))
            h = layers.Dense(hidden_dim, kernel_regularizer=reg)(inputs)
            h = layers.LeakyReLU(0.2)(h)
            h = layers.Dense(hidden_dim, kernel_regularizer=reg)(h)
            h = layers.LeakyReLU(0.2)(h)
            
            output = layers.Dense(output_dim, activation="sigmoid", kernel_regularizer=reg)(h)
            
            return tf.keras.Model(inputs, output, name=name)
    
# class AAE(tf.keras.Model):
#     def __init__(self, latent_dim):
#         super(AAE, self).__init__()
#         self.latent_dim = latent_dim
#         
#     def call(self, inputs, training=None, mask=None):
#         sequence_encoder = generator = Encoder()(inputs)
#         sequence_decoder = Decoder()(sequence_encoder)
#         return sequence_decoder

class Output(layers.Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(Output, self).__init__(**kwargs)
        
def set_trainable(model: tf.keras.Model, is_trainable):
    model.trainable = is_trainable
    for layer in model.layers:
        if type(layer) is tf.keras.Model:
            set_trainable(layer, is_trainable)
        else:
            layer.trainable = is_trainable
                
def create_adversarial_autoencoder(
        input_shape,
        classes_num=10, 
        code_len=36,
        autoencoder_optimizer=None,
        adversarial_model_optimizer=None,
        generator_optimizer=None,
        sequence_autoencoder_optimizer=None,
        autoencoder_loss='mean_squared_error',
        adversarial_model_loss='binary_crossentropy',
        generator_loss='binary_crossentropy',
        sequence_autoencoder_loss='mean_squared_error',
        generator_softmax_optimizer=None,
        generator_softmax_loss=None,
        units=256
        ):
    
    if autoencoder_optimizer is None:
        # For LSTM autoencoder use RMSprop: https://github.com/keras-team/keras/issues/1401#issuecomment-168773845
        # or Adam https://github.com/keras-team/keras/issues/1401#issuecomment-169295237
        autoencoder_optimizer = optimizers.Adam(lr=1e-5)
    if sequence_autoencoder_optimizer is None:
        sequence_autoencoder_optimizer = optimizers.Adam(lr=0.001)
    if adversarial_model_optimizer is None:
        # Use SGD for descriminator training: https://github.com/soumith/ganhacks#9-use-the-adam-optimizer
        # adversarial_model_optimizer = SGD(lr=0.01)
        adversarial_model_optimizer = optimizers.Adam(lr=1e-3, beta_1=0.1)
    if generator_optimizer is None:
        # For LSTM autoencoder use RMSprop: https://github.com/keras-team/keras/issues/1401#issuecomment-168773845
        # or Adam https://github.com/keras-team/keras/issues/1401#issuecomment-169295237
        generator_optimizer = optimizers.Adam(lr=1e-2, beta_1=0.1)
    if generator_softmax_optimizer is None:
        generator_softmax_optimizer = optimizers.Adam(lr=1e-3)
    if generator_softmax_loss is None:
        generator_softmax_loss = 'categorical_crossentropy'
    
    output_shape = input_shape
    timeseries_count=input_shape[1]
    
    sequence_encoder = generator = create_encoder(input_shape=input_shape, code_len=code_len, units=units)
    sequence_decoder = decoder = create_decoder(input_shape=generator.output_shape[1:], output_shape=output_shape, code_len=code_len+classes_num, units=units)
    inputs = generator_input = sequence_encoder.inputs
    
    # generator_softmax = create_generator_softmax(inputs, generator, classes_num=classes_num)
    
    generator_linear, generator_softmax = create_generator_linear_softmax(inputs, generator, classes_num=classes_num, code_len=code_len)
    
    generator_softmax.compile(optimizer=generator_softmax_optimizer, loss=generator_softmax_loss, metrics=['accuracy'])
    
    # generator_linear = create_generator_linear(inputs, generator, code_len=code_len)
    
    #
    # Model for reconstruction phase
    #
    
    semisupervised_autoencoder = autoencoder = create_generator_concatenate(
        inputs,
        generator_softmax,
        generator_linear,
        decoder,
        classes_num=classes_num, 
        code_len=code_len)
    autoencoder = autoencoder(inputs)
    autoencoder = tf.keras.Model(inputs=inputs, outputs=autoencoder, name='autoencoder')
    autoencoder.compile(optimizer=autoencoder_optimizer, loss=autoencoder_loss)
    semisupervised_autoencoder.compile(optimizer=autoencoder_optimizer, loss=autoencoder_loss)
    
    print("----------\nAutoencoder\n----------")
    autoencoder.summary()
    print("----------\nSemi-supervised decoder\n----------")
    semisupervised_autoencoder.summary()
    print("----------\nGenerator softmax\n----------")
    generator_softmax.summary()
    print("----------\nGenerator linear\n----------")
    generator_linear.summary()
    
    #
    # Prepare some reusable parts
    #
    
    y_onehot_fake = generator_softmax
    z_fake = generator_linear
    
    discriminator_z = create_discriminator(code_len, name='discriminator_z_output')
    discriminator_y = create_discriminator(classes_num, name='discriminator_y_output')
    
    #
    # Model for adversarial phase - update only discriminator
    #
    
    set_trainable(y_onehot_fake, False)
    set_trainable(z_fake, False)
    
    # TODO: move to separate build function
    y_onehot_true = layers.Input(y_onehot_fake.output_shape[-1:], name='y_onehot_true_input')
    # (12, )
    z_true = layers.Input(z_fake.output_shape[-1:], name='z_true_input')

    dz_fake = discriminator_z(z_fake(inputs))
    dz_fake = Output(1, name='dz_fake_output')(dz_fake)

    dy_fake = discriminator_y(y_onehot_fake(inputs))
    dy_fake = Output(1, name='dy_fake_output')(dy_fake)

    dz_true = discriminator_z(z_true)
    dz_true = Output(1, name='dz_true_output')(dz_true)

    dy_true = discriminator_y(y_onehot_true)
    dy_true = Output(1, name='dy_true_output')(dy_true)

    outputs = [dz_fake,
               dy_fake,
               dz_true,
               dy_true]

    adversarial_model = tf.keras.Model(inputs=inputs + [y_onehot_true, z_true], outputs=outputs, name='adversarial_model')

    adversarial_model.compile(optimizer=adversarial_model_optimizer, loss=adversarial_model_loss)
    
    print("----------\nAdversarial Model\n----------")
    adversarial_model.summary()
    
    #
    # Generator phase - update only encoder
    #
    
    # We will reuse discriminators in adversarial model, but we do not want to train it
    # Important that discriminator it self will be trainable, but not in scope of adversarial model
    set_trainable(discriminator_z, False)
    set_trainable(discriminator_y, False)
    set_trainable(y_onehot_fake, True)
    set_trainable(z_fake, True)
    
    # TODO: move separate build function
    
    outputs = [dz_fake, dy_fake]

    generator_model = tf.keras.Model(inputs=inputs, outputs=outputs, name='generator_model')
    generator_model.compile(optimizer=generator_optimizer, loss=generator_loss)

    print("----------\nGenerator Model\n----------")
    generator_model.summary()
    
    sequence_autoencoder = autoencoder
    
    return autoencoder, adversarial_model, generator_model, generator_softmax, generator_linear, sequence_autoencoder, semisupervised_autoencoder

# create_adversarial_autoencoder(input_shape=(251, 1, 1,))

def readucr(filename, type):
    data = np.loadtxt(os.path.join('data', 'UCR_TS_Archive_2015', filename, filename + '_' + type.upper()), delimiter = ',')
    Y = data[:,0]
    X = data[:,1:]
    return X, Y

def readsussex(type, position, sensors=None):
    if sensors is None:
        sensors = ['Acc', 'Gyr']
    data = None
    for sensor in sensors:
        for axis in ['x', 'y', 'z']:
            path = os.path.join('data', type, position, sensor + '_' + axis + '.txt')
            print('Read file %s' % (path))
            tmp = np.loadtxt(path, delimiter = ' ')
            print('Reshape')
            tmp = np.reshape(tmp, tmp.shape + (1,))
            print('Append')
            if data is None:
                data = tmp
            else:
                data = np.concatenate((data, tmp), axis=2)
    print("Size in memory %d MB" % ((data.size * data.itemsize) / 1024 / 1024))
    return data

def readsussex_labels(type, position, path = None):
    path = os.path.join('data', type, position, 'Label.txt') if path is None else path
    data = np.loadtxt(path, delimiter = ' ')
    print("Size in memory is %d MB" % ((data.size * data.itemsize) / 1024 / 1024))
    return data 
    

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline


def prepare_data(dataset_name, is_conv2d=False, feature_range=(-1, 1)):
    X_train, y_train = readucr(dataset_name, 'train')
    X_test, y_test = readucr(dataset_name, 'test')
    
    nb_classes = len(np.unique(y_test))
    # batch_size = int(min(X_train.shape[0]/10, 16))

    # y_train = np.repeat(np.reshape(y_train, (y_train.shape[0], 1, 1)), X_train.shape[1], 1)    
    # y_test = np.repeat(np.reshape(y_test, (y_test.shape[0], 1, 1)), X_train.shape[1], 1) 
    
    # y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
    # y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
    # y_train = np_utils.to_categorical(y_train, nb_classes)
    # y_test = np_utils.to_categorical(y_test, nb_classes)
    
    # X_train_mean = X_train.mean()
    # X_train_std = X_train.std()
    # X_train = (X_train - X_train_mean)/(X_train_std)
    #  
    # X_test = (X_test - X_train_mean)/(X_train_std)
    
    pipeline = Pipeline([
        # ('standard_scale', StandardScaler()),
        ('minmax_scale', MinMaxScaler(feature_range=feature_range)),
    ])
    X_train, X_test, _ = np.split(
        pipeline.fit_transform(np.concatenate((X_train, X_test), axis=0).transpose()).transpose(),
        [X_train.shape[0], X_train.shape[0] + X_test.shape[0]]
    )
    
    X_train_new_shape = X_train.shape + ((1,1,) if is_conv2d else (1,))
    X_test_new_shape = X_test.shape + ((1,1,) if is_conv2d else (1,))
    
    X_train = X_train.reshape(X_train_new_shape)
    X_test = X_test.reshape(X_test_new_shape)
    
    input_shape = X_train.shape[1:]
    
    return input_shape, X_train, y_train, X_test, y_test

def prepare_sussex_data(
        batch_size,
        feature_range=(-1, 1),
        test_size=0.66,
        include_test=False,
):
    x_validate_hand = np.load('data/validate_Hand_AG.npy')#[:, :, :1]
    y_validate_hand = np.load('data/validate_Hand_Label.npy')
    y_validate_hand = AaeClustering.get_labels_for_segments(
        np.reshape(y_validate_hand, y_validate_hand.shape + (1,))
    ).astype(int)
    # make class numbers starting from 0
    y_validate_hand -= 1
    x_test_hand = np.load('data/test_Hand_AG.npy')#[:, :, :1]
    
    x_train, x_test, y_train, y_test = train_test_split(
        x_validate_hand,
        y_validate_hand,
        test_size=test_size,
        random_state=0,
        stratify=y_validate_hand
    )
    
    # normalize time series
    pipeline = Pipeline([
        #('minmax_scale', MinMaxScaler(feature_range=feature_range)),
        ('standard_scale', StandardScaler()),
    ])
    for axis in range(x_train.shape[2]):
        x_train[:, :, axis] = pipeline.fit_transform(x_train[:, :, axis])
        x_test[:, :, axis] = pipeline.fit_transform(x_test[:, :, axis])
        x_test_hand[:, :, axis] = pipeline.fit_transform(x_test_hand[:, :, axis])
    
    classes_num = np.unique(y_validate_hand).shape[0]
    
    test_data = (x_test, y_test)
    if include_test:
        test_data = (
            np.concatenate((x_test, x_test_hand), axis=0), 
            np.concatenate((y_test, np.zeros(x_test_hand.shape[0], int)))
        )
    
    dataset = Dataset(
        (x_train, y_train),
        test_data,
        num_labeled_data=x_train.shape[0], 
        num_classes=classes_num,
        use_test_as_unlabeled=True,
        batch_size=batch_size)
    dataset.summary()
    
    return dataset, x_train, y_train, x_test, y_test, classes_num

def prepare_sussex_test_data():
    x_validate_hand = np.load('data/validate_Hand_AG.npy')#[:, :, :1]
    y_validate_hand = np.load('data/validate_Hand_Label.npy')
    y_validate_hand = AaeClustering.get_labels_for_segments(
        np.reshape(y_validate_hand, y_validate_hand.shape + (1,))
    ).astype(int)
    # make class numbers starting from 0
    y_validate_hand -= 1
    x_test = np.load('data/test_Hand_AG.npy')#[:, :, :1]
    
    x_train, y_train = x_validate_hand, y_validate_hand
    
    # normalize time series
    pipeline = Pipeline([
        #('minmax_scale', MinMaxScaler(feature_range=feature_range)),
        ('standard_scale', StandardScaler()),
    ])
    for axis in range(x_train.shape[2]):
        x_train[:, :, axis] = pipeline.fit_transform(x_train[:, :, axis])
        x_test[:, :, axis] = pipeline.fit_transform(x_test[:, :, axis])
    
    classes_num = np.unique(y_validate_hand).shape[0]
        
    return x_train, y_train, x_test, classes_num

In [ ]:
dataset, x_train, y_train, x_test, y_test, classes_num = prepare_sussex_data(1000)

In [ ]:
from math import ceil
from tensorflow.python.keras.callbacks import CallbackList
from timeseries.profiler import Stopwatch

def train_aae(
        classes_num=3,
        code_len=36,
        labeled_data_num=11,
        batchsize=None,
        epochs=500,
        verbose=True,
        wgan=True,
        adversarial_model_loss='binary_crossentropy',
        generator_loss='binary_crossentropy',
        evaluation_step=5,
        profiling=False
):
    stopwatch = Stopwatch(disable=(not profiling))
    input_shape, X_train, y_train, X_test, y_test = prepare_data('ArrowHead')
    
    if wgan:
        # adversarial_model_loss = self.wasserstein_loss
        # generator_loss = self.wasserstein_loss
        adversarial_model_loss = wasserstein_discriminator_loss
        generator_loss = wasserstein_generator_loss
        
    # Create and compile model
    autoencoder, adversarial_model, generator_model, generator_softmax, generator_linear, sequence_autoencoder, semisupervised_autoencoder = create_adversarial_autoencoder(
        input_shape,
        classes_num=classes_num,
        code_len=code_len,
        adversarial_model_loss=adversarial_model_loss,
        generator_loss=generator_loss,
    )
    
    exclude_models_by_name = []
    plot_model_recursive(adversarial_model, exclude_models_by_name=exclude_models_by_name)
    plot_model_recursive(autoencoder, exclude_models_by_name=exclude_models_by_name)
            
    dataset = Dataset(
        (X_train, AaeClustering.get_labels_for_segments(y_train).astype(int)),
        (X_test, AaeClustering.get_labels_for_segments(y_test).astype(int)),
        num_labeled_data=labeled_data_num, 
        num_classes=classes_num,
        use_test_as_unlabeled=True,
        batch_size=batchsize)
    dataset.summary()
    batchsize = dataset.batch_size
    steps = dataset.steps

    progbar = tf.keras.callbacks.ProgbarLogger(count_mode='steps')
    tensorboard = tf.keras.callbacks.TensorBoard(
        log_dir='./tb_logs/adversarial-autoencoder-' + datetime.datetime.today().strftime('%Y%m%dT%H%M'),
        histogram_freq=False,
        # histogram_freq=20,
        write_graph=True,
        # TODO: this option might affect logs size
        write_grads=False
    )
    
    callbacks = CallbackList([
        progbar,
        tensorboard])
    callbacks.set_params({
        'epochs': epochs,
        'verbose': verbose,
        'steps': steps,
        'metrics': []})
    callbacks.set_model(autoencoder)
    
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint('saved_models/aae_generator_softmax-{epoch:02d}.hdf5', monitor='accuracy', verbose=1)
    model_checkpoint.set_model(generator_softmax)
    callbacks.append(model_checkpoint)
    
    autoencoder_out_labels = ['mse']
    sequence_autoencoder_out_labels = ['mse']
    
    callbacks.on_train_begin()
    
    for epoch in np.arange(0, epochs):
        epoch_logs = {}
        
        callbacks.on_epoch_begin(epoch)
        
        # dataset should be shuffled on each epoch
        dataset.shuffle()
        
        # TODO: add learning rate adjustments
        
        for index in range(0, steps):
            
            X_train_batch = dataset.sample_unlabeled_minibatch(batchsize)
            if labeled_data_num > 0:
                X_train_batch_labeled, y_l, y_onehot_l = dataset.sample_labeled_minibatch(batchsize)
            actual_batch_size = X_train_batch.shape[0]
            
            batch_logs = {'batch': index, 'size': actual_batch_size}
            
            callbacks.on_batch_begin(index)
            
            # reconstruction phase
            stopwatch.start('reconstruction phase')
            autoencoder_outs = autoencoder.train_on_batch(X_train_batch, X_train_batch)
            batch_logs = AdversarialAutoencoder.get_logs(batch_logs, autoencoder_outs, autoencoder_out_labels)
            stopwatch.stop('reconstruction phase')
            
            # if labeled_data_num > 0:
            #     stopwatch.start('semisupervised phase')
            #     autoencoder_outs = semisupervised_autoencoder.train_on_batch([X_train_batch_labeled, y_onehot_l], X_train_batch_labeled)
            #     batch_logs = AdversarialAutoencoder.get_logs(batch_logs, autoencoder_outs, autoencoder_out_labels)
            #     stopwatch.stop('semisupervised phase')
            
            # adversarial phase
            stopwatch.start('adversarial phase')
            # TODO: might be vise versa https://github.com/eriklindernoren/Keras-GAN/blob/master/aae/aae.py#L120
            if wgan:
                class_true = np.ones(actual_batch_size, dtype=np.int32)
                class_fake = -class_true
            else:
                class_fake = np.zeros(actual_batch_size, dtype=np.int32)
                class_true = np.ones(actual_batch_size, dtype=np.int32)
                
            adversarial_input = {'encoder_input': X_train_batch,
                                 'z_true_input': AdversarialAutoencoder.gaussian(actual_batch_size, code_len),
                                 'y_onehot_true_input': AdversarialAutoencoder.onehot_categorical(actual_batch_size, classes_num)}
            adversarial_output = {'dz_fake_output': class_fake,
                                  'dy_fake_output': class_fake,
                                  'dz_true_output': class_true,
                                  'dy_true_output': class_true}
            
            adversarial_model.train_on_batch(adversarial_input, adversarial_output)
            
            # generator phase
            generator_input = {'encoder_input': X_train_batch}
            generator_output = {'dz_fake_output': class_true,
                                'dy_fake_output': class_true}

            generator_model.train_on_batch(generator_input, generator_output)
            stopwatch.stop('adversarial phase')
            
            # supervised phase
            stopwatch.start('supervised phase')
            if labeled_data_num > 0:
                autoencoder_outs = generator_softmax.train_on_batch(X_train_batch_labeled, y_onehot_l)
                batch_logs = AdversarialAutoencoder.get_logs(batch_logs, autoencoder_outs, autoencoder_out_labels)
            stopwatch.stop('supervised phase')
            
            # TODO: there was progbar instead callbacks. It might affect logs size
            callbacks.on_batch_end(index, batch_logs)
            
        #
        # Evaluate autoencoder
        #

        if epoch % evaluation_step == 0:
            stopwatch.start('evaluate phase')
            def evaluate_autoencoder(epoch_logs, X_train, logs_name):
                autoencoder_outs = autoencoder.evaluate(
                    X_train, X_train,
                    batch_size=batchsize,
                    verbose=0)
                return AdversarialAutoencoder.get_logs(epoch_logs, autoencoder_outs, [logs_name])
    
            epoch_logs = evaluate_autoencoder(epoch_logs, X_train, 'autoencoder_loss')
            epoch_logs = evaluate_autoencoder(epoch_logs, X_test, 'autoencoder_loss_test')
    
            #
            # Evaluate adversarial model
            #
    
            class_true = np.zeros(X_train.shape[0], dtype=np.int32)
            class_fake = np.ones(X_train.shape[0], dtype=np.int32)
            class_true_test = np.zeros(X_test.shape[0], dtype=np.int32)
            class_fake_test = np.ones(X_test.shape[0], dtype=np.int32)
    
            def evaluate_adversarial_model(epoch_logs, X_train, code_len, class_true, class_fake, logs_name):
                adversarial_input = {'encoder_input': X_train,
                                     'z_true_input': AdversarialAutoencoder.gaussian(X_train.shape[0], code_len),
                                     # TODO: fix errors in some cases. You might use `code_len` instead of `self.classes_num`
                                     'y_onehot_true_input': AdversarialAutoencoder.onehot_categorical(X_train.shape[0], classes_num)}
                adversarial_output = {'dz_fake_output': class_fake,
                                      'dy_fake_output': class_fake,
                                      'dz_true_output': class_true,
                                      'dy_true_output': class_true
                                      }
                adversarial_model_outs = adversarial_model.evaluate(
                    adversarial_input, adversarial_output,
                    batch_size=batchsize,
                    verbose=0)
                return AdversarialAutoencoder.get_logs(epoch_logs, adversarial_model_outs, [logs_name])
    
            epoch_logs = evaluate_adversarial_model(epoch_logs, X_train, code_len, class_true, class_fake, 'adversarial_loss')
            epoch_logs = evaluate_adversarial_model(epoch_logs, X_test, code_len, class_true_test, class_fake_test, 'adversarial_loss_test')
    
            #
            # Generator phase
            #
    
            # train
            def evaluate_generator(epoch_logs, X, class_true, logs_name):
                generator_input = {'encoder_input': X}
                generator_output = {'dz_fake_output': class_true,
                                    'dy_fake_output': class_true}
                generator_model_outs = generator_model.evaluate(
                    generator_input, generator_output,
                    batch_size=batchsize,
                    verbose=0)
                return AdversarialAutoencoder.get_logs(epoch_logs, generator_model_outs, [logs_name])
    
            epoch_logs = evaluate_generator(epoch_logs, X_train, class_true, 'generator_loss')
            epoch_logs = evaluate_generator(epoch_logs, X_test, class_true_test, 'generator_loss_test')
    
            # eval clustering
            epoch_logs = AdversarialAutoencoder.get_logs(epoch_logs, [AaeClustering.evaluate(X_train, y_train, generator_softmax)], ['adjusted_rand_score'])
            epoch_logs = AdversarialAutoencoder.get_logs(epoch_logs, [AaeClustering.evaluate(X_test, y_test, generator_softmax)], ['adjusted_rand_score_test'])
            epoch_logs = AdversarialAutoencoder.get_logs(epoch_logs, [AaeClustering.evaluate(X_train, y_train, generator_softmax, metric_func=accuracy_score)], ['accuracy_score'])
            epoch_logs = AdversarialAutoencoder.get_logs(epoch_logs, [AaeClustering.evaluate(X_test, y_test, generator_softmax, metric_func=accuracy_score)], ['accuracy_score_test'])
            metric_func = lambda y_true, y_pred: f1_score(y_true, y_pred, average='macro')
            epoch_logs = AdversarialAutoencoder.get_logs(epoch_logs, [AaeClustering.evaluate(X_train, y_train, generator_softmax, metric_func=metric_func)], ['f1_score'])
            epoch_logs = AdversarialAutoencoder.get_logs(epoch_logs, [AaeClustering.evaluate(X_test, y_test, generator_softmax, metric_func=metric_func)], ['f1_score_test'])

            stopwatch.stop('evaluate phase')
            
        callbacks.on_epoch_end(epoch, epoch_logs)
            
    callbacks.on_train_end()
    
    sequence_autoencoder = autoencoder
    
    # TODO: implement plot prediction
    # self.plotPrediction(data, sequence_autoencoder)
    
    predictions = autoencoder.predict(X_test[:1])
    for axis in range(predictions.shape[2]):
        plt.plot(np.reshape(X_test[:1, :, axis:axis+1], (X_test.shape[1])))
        plt.plot(np.reshape(predictions[:, :, axis:axis+1], (predictions.shape[1])))
        plt.show()
    
    return generator_softmax, sequence_autoencoder, autoencoder, adversarial_model, generator_model, generator_linear

train_aae()

In [3]:
from math import ceil
from tensorflow.python.keras.callbacks import CallbackList
from timeseries.data import Dataset
from timeseries.transformers import AaeClustering, AdversarialAutoencoder
from tensorflow.contrib.gan.python.losses.python.losses_impl import wasserstein_discriminator_loss, wasserstein_generator_loss
from timeseries.profiler import Stopwatch
from sklearn.metrics import accuracy_score

def train_sussex_aae(
        code_len=50,
        batchsize=50,
        epochs=50,
        verbose=True,
        wgan=True,
        adversarial_model_loss='binary_crossentropy',
        generator_loss='binary_crossentropy',
        evaluation_step=1,
        profiling=False,
        units=512,
        test_size=0.66,
        include_test=True,
        experiment_name=None,
):
    if experiment_name is None:
        experiment_name = '{:.5}'.format(str(abs(hash(frozenset(
            {'code_len':code_len, 'batchsize': batchsize, 'units': 
                units, 'test_size': test_size, 'include_test': include_test}.items()
        )))))
        
    print('Experiment name: %s' % (experiment_name,))
    
    stopwatch = Stopwatch(disable=(not profiling))
    
    dataset, X_train, y_train, X_test, y_test, classes_num = prepare_sussex_data(batchsize, test_size=test_size, include_test=include_test)
    batchsize = dataset.batch_size
    steps = dataset.steps
    labeled_data_num = dataset.get_num_labeled_data()
    
    input_shape = X_train.shape[1:]
    
    if wgan:
        # adversarial_model_loss = self.wasserstein_loss
        # generator_loss = self.wasserstein_loss
        adversarial_model_loss = wasserstein_discriminator_loss
        generator_loss = wasserstein_generator_loss
        
    # Create and compile model
    autoencoder, adversarial_model, generator_model, generator_softmax, generator_linear, sequence_autoencoder, semisupervised_autoencoder = create_adversarial_autoencoder(
        input_shape,
        classes_num=classes_num,
        code_len=code_len,
        adversarial_model_loss=adversarial_model_loss,
        generator_loss=generator_loss,
        units=units
    )
    
    exclude_models_by_name = []
    plot_model_recursive(adversarial_model, exclude_models_by_name=exclude_models_by_name)
    plot_model_recursive(autoencoder, exclude_models_by_name=exclude_models_by_name)

    progbar = tf.keras.callbacks.ProgbarLogger(count_mode='steps')
    tensorboard = tf.keras.callbacks.TensorBoard(
        log_dir='./tb_logs/adversarial-autoencoder-' + experiment_name + '-' + datetime.datetime.today().strftime('%Y%m%dT%H%M'),
        histogram_freq=False,
        # histogram_freq=20,
        write_graph=True,
        # TODO: this option might affect logs size
        write_grads=False
    )
    
    callbacks = CallbackList([
        progbar,
        tensorboard])
    callbacks.set_params({
        'epochs': epochs,
        'verbose': verbose,
        'steps': steps,
        'metrics': []})
    callbacks.set_model(autoencoder)
    
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint('saved_models/sussex_aae_generator_softmax-' + experiment_name + '-{epoch:02d}.hdf5', monitor='accuracy', verbose=1)
    model_checkpoint.set_model(generator_softmax)
    callbacks.append(model_checkpoint)
    
    autoencoder_out_labels = ['mse']
    sequence_autoencoder_out_labels = ['mse']
    
    callbacks.on_train_begin()
    
    for epoch in np.arange(0, epochs):
        epoch_logs = {}
        
        callbacks.on_epoch_begin(epoch)
        
        # dataset should be shuffled on each epoch
        dataset.shuffle()
        
        # TODO: add learning rate adjustments
        
        for index in range(0, steps):
            
            X_train_batch = dataset.sample_unlabeled_minibatch(batchsize)
            if labeled_data_num > 0:
                X_train_batch_labeled, y_l, y_onehot_l = dataset.sample_labeled_minibatch(batchsize)
            actual_batch_size = X_train_batch.shape[0]
            
            batch_logs = {'batch': index, 'size': actual_batch_size}
            
            callbacks.on_batch_begin(index)
            
            # reconstruction phase
            stopwatch.start('reconstruction phase')
            autoencoder_outs = autoencoder.train_on_batch(X_train_batch, X_train_batch)
            batch_logs = AdversarialAutoencoder.get_logs(batch_logs, autoencoder_outs, autoencoder_out_labels)
            stopwatch.stop('reconstruction phase')
            
            # if labeled_data_num > 0:
            #     stopwatch.start('semisupervised phase')
            #     autoencoder_outs = semisupervised_autoencoder.train_on_batch([X_train_batch_labeled, y_onehot_l], X_train_batch_labeled)
            #     batch_logs = AdversarialAutoencoder.get_logs(batch_logs, autoencoder_outs, autoencoder_out_labels)
            #     stopwatch.stop('semisupervised phase')
            
            # adversarial phase
            stopwatch.start('adversarial phase')
            # TODO: might be vise versa https://github.com/eriklindernoren/Keras-GAN/blob/master/aae/aae.py#L120
            if wgan:
                class_true = np.ones(actual_batch_size, dtype=np.int32)
                class_fake = -class_true
            else:
                class_fake = np.zeros(actual_batch_size, dtype=np.int32)
                class_true = np.ones(actual_batch_size, dtype=np.int32)
                
            adversarial_input = {'encoder_input': X_train_batch,
                                 'z_true_input': AdversarialAutoencoder.gaussian(actual_batch_size, code_len),
                                 'y_onehot_true_input': AdversarialAutoencoder.onehot_categorical(actual_batch_size, classes_num)}
            adversarial_output = {'dz_fake_output': class_fake,
                                  'dy_fake_output': class_fake,
                                  'dz_true_output': class_true,
                                  'dy_true_output': class_true}
            
            adversarial_model.train_on_batch(adversarial_input, adversarial_output)
            
            # generator phase
            generator_input = {'encoder_input': X_train_batch}
            generator_output = {'dz_fake_output': class_true,
                                'dy_fake_output': class_true}

            generator_model.train_on_batch(generator_input, generator_output)
            stopwatch.stop('adversarial phase')
            
            # supervised phase
            stopwatch.start('supervised phase')
            if labeled_data_num > 0:
                autoencoder_outs = generator_softmax.train_on_batch(X_train_batch_labeled, y_onehot_l)
                batch_logs = AdversarialAutoencoder.get_logs(batch_logs, autoencoder_outs, autoencoder_out_labels)
            stopwatch.stop('supervised phase')
            
            # TODO: there was progbar instead callbacks. It might affect logs size
            callbacks.on_batch_end(index, batch_logs)
            
        #
        # Evaluate autoencoder
        #

        if epoch % evaluation_step == 0:
            stopwatch.start('evaluate phase')
            def evaluate_autoencoder(epoch_logs, X_train, logs_name):
                autoencoder_outs = autoencoder.evaluate(
                    X_train, X_train,
                    batch_size=batchsize,
                    verbose=0)
                return AdversarialAutoencoder.get_logs(epoch_logs, autoencoder_outs, [logs_name])
    
            epoch_logs = evaluate_autoencoder(epoch_logs, X_train, 'autoencoder_loss')
            epoch_logs = evaluate_autoencoder(epoch_logs, X_test, 'autoencoder_loss_test')
    
            #
            # Evaluate adversarial model
            #
    
            class_true = np.zeros(X_train.shape[0], dtype=np.int32)
            class_fake = np.ones(X_train.shape[0], dtype=np.int32)
            class_true_test = np.zeros(X_test.shape[0], dtype=np.int32)
            class_fake_test = np.ones(X_test.shape[0], dtype=np.int32)
    
            def evaluate_adversarial_model(epoch_logs, X_train, code_len, class_true, class_fake, logs_name):
                adversarial_input = {'encoder_input': X_train,
                                     'z_true_input': AdversarialAutoencoder.gaussian(X_train.shape[0], code_len),
                                     # TODO: fix errors in some cases. You might use `code_len` instead of `self.classes_num`
                                     'y_onehot_true_input': AdversarialAutoencoder.onehot_categorical(X_train.shape[0], classes_num)}
                adversarial_output = {'dz_fake_output': class_fake,
                                      'dy_fake_output': class_fake,
                                      'dz_true_output': class_true,
                                      'dy_true_output': class_true
                                      }
                adversarial_model_outs = adversarial_model.evaluate(
                    adversarial_input, adversarial_output,
                    batch_size=batchsize,
                    verbose=0)
                return AdversarialAutoencoder.get_logs(epoch_logs, adversarial_model_outs, [logs_name])
    
            epoch_logs = evaluate_adversarial_model(epoch_logs, X_train, code_len, class_true, class_fake, 'adversarial_loss')
            epoch_logs = evaluate_adversarial_model(epoch_logs, X_test, code_len, class_true_test, class_fake_test, 'adversarial_loss_test')
    
            #
            # Generator phase
            #
    
            # train
            def evaluate_generator(epoch_logs, X, class_true, logs_name):
                generator_input = {'encoder_input': X}
                generator_output = {'dz_fake_output': class_true,
                                    'dy_fake_output': class_true}
                generator_model_outs = generator_model.evaluate(
                    generator_input, generator_output,
                    batch_size=batchsize,
                    verbose=0)
                return AdversarialAutoencoder.get_logs(epoch_logs, generator_model_outs, [logs_name])
    
            epoch_logs = evaluate_generator(epoch_logs, X_train, class_true, 'generator_loss')
            epoch_logs = evaluate_generator(epoch_logs, X_test, class_true_test, 'generator_loss_test')
    
            # eval clustering
            epoch_logs = AdversarialAutoencoder.get_logs(epoch_logs, [AaeClustering.evaluate(X_train, y_train, generator_softmax)], ['adjusted_rand_score'])
            epoch_logs = AdversarialAutoencoder.get_logs(epoch_logs, [AaeClustering.evaluate(X_test, y_test, generator_softmax)], ['adjusted_rand_score_test'])
            epoch_logs = AdversarialAutoencoder.get_logs(epoch_logs, [AaeClustering.evaluate(X_train, y_train, generator_softmax, metric_func=accuracy_score)], ['accuracy_score'])
            epoch_logs = AdversarialAutoencoder.get_logs(epoch_logs, [AaeClustering.evaluate(X_test, y_test, generator_softmax, metric_func=accuracy_score)], ['accuracy_score_test'])
            metric_func = lambda y_pred, y_true: f1_score(y_pred, y_true, average='macro')
            epoch_logs = AdversarialAutoencoder.get_logs(epoch_logs, [AaeClustering.evaluate(X_train, y_train, generator_softmax, metric_func=metric_func)], ['f1_score'])
            epoch_logs = AdversarialAutoencoder.get_logs(epoch_logs, [AaeClustering.evaluate(X_test, y_test, generator_softmax, metric_func=metric_func)], ['f1_score_test'])

            stopwatch.stop('evaluate phase')
            
        callbacks.on_epoch_end(epoch, epoch_logs)
            
    callbacks.on_train_end()
    
    sequence_autoencoder = autoencoder
    
    # TODO: implement plot prediction
    # self.plotPrediction(data, sequence_autoencoder)
    
    predictions = autoencoder.predict(X_test[:1])
    for axis in range(predictions.shape[2]):
        plt.plot(np.reshape(X_test[:1, :, axis:axis+1], (X_test.shape[1])))
        plt.plot(np.reshape(predictions[:, :, axis:axis+1], (predictions.shape[1])))
        plt.show()
    
    return generator_softmax, sequence_autoencoder, autoencoder, adversarial_model, generator_model, generator_linear

# Experiment w/o test data and large batchsize
train_sussex_aae(batchsize=500, include_test=False)
# Experiment w/o test data
train_sussex_aae(batchsize=50, include_test=False)
# Final experiment
train_sussex_aae(batchsize=50, include_test=True)

In [ ]:
def create_autoencoder(
        input_shape,
        code_len,
        name='autoencoder'
):
    output_shape = input_shape
    encoder = create_encoder(input_shape=input_shape, code_len=code_len)
    display(encoder.output_shape)
    decoder = create_decoder(
        input_shape=encoder.output_shape[1:], 
        output_shape=output_shape,
        code_len=code_len)
    
    autoencoder_optimizer = optimizers.Adam(lr=0.001)
    autoencoder_loss='mean_squared_error'
    
    autoencoder = tf.keras.Model(encoder.inputs, decoder(encoder(encoder.inputs)), name=name)
    autoencoder.compile(
        optimizer=autoencoder_optimizer, 
        loss=autoencoder_loss,)
    
    return autoencoder
    
def train_autoencoder(epochs=500, verbose=True, batch_size=36):
    input_shape, X_train, y_train, X_test, y_test = prepare_data('ArrowHead')
    
    display(input_shape)
    
    autoencoder = create_autoencoder(input_shape, code_len=36)
    plot_model_recursive(autoencoder)
    
    tensorboard = tf.keras.callbacks.TensorBoard(
        log_dir='./tb_logs/cnn_autoencoder-' + datetime.datetime.today().strftime('%Y%m%dT%H%M'),
        histogram_freq=False,
        # histogram_freq=20,
        write_graph=True,
        # TODO: this option might affect logs size
        write_grads=False
    )

    history = autoencoder.fit(
        X_train, 
        X_train,
        epochs=epochs,
        verbose=verbose,
        batch_size=batch_size,
        shuffle=True,
        validation_data=(X_test, X_test),
        callbacks=[
            tensorboard,
        ]
    )
    
    predictions = autoencoder.predict(X_test[:1])
    plt.plot(np.reshape(X_test[:1], (X_test.shape[1])))
    plt.plot(np.reshape(predictions, (predictions.shape[1])))
    
train_autoencoder()

In [ ]:
def create_autoencoder_multivariate(
        input_shape,
        code_len,
        classes_num,
        name='autoencoder_multivariate',
        units=512,
):
    adversarial_model_loss = wasserstein_discriminator_loss
    generator_loss = wasserstein_generator_loss
        
    # Create and compile model
    autoencoder, _, _, _, _, _, _ = create_adversarial_autoencoder(
        input_shape,
        classes_num=classes_num,
        code_len=code_len,
        adversarial_model_loss=adversarial_model_loss,
        generator_loss=generator_loss,
        units=units,
        autoencoder_optimizer=optimizers.Adam(lr=1e-4),
    )
    
    return autoencoder
    
    
def train_autoencoder_multivariate(epochs=10, verbose=True, batchsize=50, code_len=50, test_size=0.33, include_test=False):
    
    dataset, X_train, y_train, X_test, y_test, classes_num = prepare_sussex_data(batchsize, test_size=test_size, include_test=include_test)
    batchsize = dataset.batch_size
    steps = dataset.steps
    labeled_data_num = dataset.get_num_labeled_data()
    input_shape = X_train.shape[1:]
    
    display(input_shape)
    
    autoencoder = create_autoencoder_multivariate(input_shape, code_len=code_len, classes_num=classes_num)
    plot_model_recursive(autoencoder)
    
    tensorboard = tf.keras.callbacks.TensorBoard(
        log_dir='./tb_logs/cnn_autoencoder_multivariate-' + datetime.datetime.today().strftime('%Y%m%dT%H%M'),
        histogram_freq=False,
        # histogram_freq=20,
        write_graph=True,
        # TODO: this option might affect logs size
        write_grads=False
    )

    history = autoencoder.fit(
        X_train, 
        X_train,
        epochs=epochs,
        verbose=verbose,
        batch_size=batchsize,
        shuffle=True,
        validation_data=(X_test, X_test),
        callbacks=[
            tensorboard,
        ]
    )
    
    predictions = autoencoder.predict(X_test[:1])
    for axis in range(predictions.shape[2]):
        plt.plot(np.reshape(X_test[:1, :, axis:axis+1], (X_test.shape[1])))
        plt.plot(np.reshape(predictions[:, :, axis:axis+1], (predictions.shape[1])))
        plt.show()
        
    return autoencoder
    
train_autoencoder_multivariate()

In [ ]:
def create_supervised_cnn_classifier(
        input_shape,
        code_len,
        classes_num,
        name='supervised_cnn_classifier',
):
    generator = create_encoder(input_shape=input_shape, code_len=code_len)
    inputs = generator.inputs
        
    _, classifier = create_generator_linear_softmax(inputs, generator, classes_num=classes_num, code_len=code_len, name=name)
        
    classifier.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy'])
    
    return classifier

def train_supervised_cnn_classifier():
    nb_epochs = 1000
    nb_classes = 3
    
    fname = 'ArrowHead'
    input_shape, X_train, y_train, X_test, y_test = prepare_data(fname)
    
    y_train = np_utils.to_categorical(y_train, nb_classes)
    y_test = np_utils.to_categorical(y_test, nb_classes)
    
    batch_size = int(min(X_train.shape[0]/10, 16))
    
    model = create_supervised_cnn_classifier(input_shape, code_len=36, classes_num=nb_classes)
    plot_model_recursive(model)
    
    reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor=0.5,
                  patience=50, min_lr=0.0001) 
    tb_callback = TensorBoard(
                log_dir='./tb_logs/supervised_cnn_classifier-' + fname + '-' + datetime.datetime.today().strftime('%Y%m%dT%H%M'),
                histogram_freq=100,
                write_graph=True,
                # TODO: this option might affect logs size
                write_grads=False)
    hist = model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epochs,
              verbose=1, validation_data=(X_test, y_test), callbacks=[reduce_lr, tb_callback])
    #Print the testing results which has the lowest training loss.
    log = pd.DataFrame(hist.history)
    print(log.loc[log['loss'].idxmin]['loss'], log.loc[log['loss'].idxmin]['val_acc'])
    K.clear_session()
    
train_supervised_cnn_classifier()

In [ ]:
def create_supervised_cnn_classifier2(
        input_shape,
        code_len,
        classes_num,
        name='supervised_cnn_classifier',
):
    adversarial_model_loss = wasserstein_discriminator_loss
    generator_loss = wasserstein_generator_loss
        
    # Create and compile model
    _, _, _, classifier, _, _, _ = create_adversarial_autoencoder(
        input_shape,
        classes_num=classes_num,
        code_len=code_len,
        adversarial_model_loss=adversarial_model_loss,
        generator_loss=generator_loss,
    )
    
    return classifier

def train_supervised_cnn_classifier2():
    nb_epochs = 1000
    nb_classes = 3
    
    fname = 'ArrowHead'
    input_shape, X_train, y_train, X_test, y_test = prepare_data(fname)
    
    y_train = np_utils.to_categorical(y_train, nb_classes)
    y_test = np_utils.to_categorical(y_test, nb_classes)
    
    batch_size = int(min(X_train.shape[0]/10, 16))
    
    model = create_supervised_cnn_classifier2(input_shape, code_len=36, classes_num=nb_classes)
    plot_model_recursive(model)
    
    reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor=0.5,
                  patience=50, min_lr=0.0001) 
    tb_callback = TensorBoard(
                log_dir='./tb_logs/supervised_cnn_classifier2-' + fname + '-' + datetime.datetime.today().strftime('%Y%m%dT%H%M'),
                histogram_freq=100,
                write_graph=True,
                # TODO: this option might affect logs size
                write_grads=False)
    hist = model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epochs,
              verbose=1, validation_data=(X_test, y_test), callbacks=[reduce_lr, tb_callback])
    #Print the testing results which has the lowest training loss.
    log = pd.DataFrame(hist.history)
    print(log.loc[log['loss'].idxmin]['loss'], log.loc[log['loss'].idxmin]['val_acc'])
    K.clear_session()
    
train_supervised_cnn_classifier2()

In [3]:
def test_sussex(
        generator_softmax_path='saved_models/sussex_aae_generator_softmax-47.hdf5',
        epochs=10, verbose=True, batchsize=50, code_len=50, test_size=0.66, include_test=True
):
    x_train, y_train, x_test, classes_num = prepare_sussex_test_data()

    input_shape = x_train.shape[1:]
    adversarial_model_loss = wasserstein_discriminator_loss
    generator_loss = wasserstein_generator_loss
        
    # Create and compile model
#     _, _, _, classifier, _, _, _ = create_adversarial_autoencoder(
#         input_shape,
#         classes_num=classes_num,
#         code_len=code_len,
#         adversarial_model_loss=adversarial_model_loss,
#         generator_loss=generator_loss,
#         units=512
#     )
    
#     classifier.load_weights(generator_softmax_path)
    # FIXME: https://github.com/tensorflow/tensorflow/issues/27769
    classifier = tf.keras.models.load_model(generator_softmax_path)
    
    metric_func = lambda y_pred, y_true: f1_score(y_pred, y_true, average='macro')
#     display(AaeClustering.evaluate(x_train, y_train, classifier, metric_func=metric_func))
    #AaeClustering.evaluate(x_test, y_test, classifier, metric_func=metric_func)
    
    w_pred = classifier.predict(x_test)
    y_pred = AaeClustering.get_labels_for_softmax(w_pred)
    display(x_test.shape)
    display(y_pred.shape)
    np.savetxt('data/DB_prediction.txt',
               np.repeat(np.reshape(y_pred, (y_pred.shape[0],1)), x_test.shape[1], axis=1),
               fmt='%i',
               delimiter=" ")
    
test_sussex()

In [4]:
%%time
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import matplotlib.colors as mcolors

def plot_confusion_matrix(y_true, y_pred, classes,
                          labels, dpi=300):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
            
    # combine a new colormap
    colors = np.vstack(([[1, 1, 1, 1,]], plt.cm.Reds(np.linspace(0,1.,127)), plt.cm.Greens(np.linspace(0,1.,128))))
    cmap = mcolors.LinearSegmentedColormap.from_list('confusion_matrix_colormap', colors)

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes[np.where(classes == unique_labels(y_true, y_pred))]
    
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100.0
    print("Normalized confusion matrix")

    print(cm)

    fig, ax = plt.subplots(dpi=dpi)
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    #ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=labels, yticklabels=labels,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '{0:.0f}%'
    # TODO: fix the thresh
    thresh = cm.max() / 2.
    display(cm.shape, thresh)
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            if cm[i, j] >= 1.0:
                text = fmt.format(cm[i, j])
            elif cm[i, j] == 0.0:
                text = ''
            else:
                text = '<1%'
            ax.text(j, i, text,
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    
    print('F1-score = %s' % f1_score(y_true, y_pred, average='macro'))
    
    plt.savefig('plots/confusion_matrix.png', dpi=dpi)
    
    plt.show()
    return ax

def validate_sussex(
        generator_softmax_path='saved_models/sussex_aae_generator_softmax-47.hdf5',
        test_size=0.66,
        batchsize=50,
):
    # x_train, y_train, x_test, classes_num = prepare_sussex_test_data()
    _, _, _, x_test, y_test, _ = prepare_sussex_data(batchsize, test_size=test_size, include_test=False)

    classifier = tf.keras.models.load_model(generator_softmax_path)
        
    w_pred = classifier.predict(x_test)
    y_pred = AaeClustering.get_labels_for_softmax(w_pred).astype(int)
    
    y_true = y_test
    
    display(y_pred.shape)
    display(y_true.shape)

    classes = np.array([0, 1, 2, 3, 4, 5, 6, 7])
    plot_confusion_matrix(y_true, y_pred, classes, labels=['1', '2', '3', '4', '5', '6', '7', '8']) 
    
validate_sussex()

# Best models
46 => 0.88

50 => 0.9239 

49 => 0.9208 

48 => 0.9170 

47 => 0.9320 (best model !!!)

45 => 0.9255 
